In [2]:
from zipfile import ZipFile

#filename = 'race_20452'
filename = 'race_20409'
dirname = 'race/' + filename.split('_')[1]

zf = ZipFile(filename + '.zip', 'r')
zf.extractall(dirname)
zf.close()

In [33]:
import pandas as pd
from glob import glob
import io
import numpy as np

def angle_from_vector_to_vector(a,b):
  a = np.deg2rad(a)
  b = np.deg2rad(b)
  vb = [np.cos(a), np.sin(a)]
  vv = [np.cos(b), np.sin(b)]

  unit_vector_1 = vb / np.linalg.norm(vb)
  unit_vector_2 = vv / np.linalg.norm(vv)
  dot_product = np.dot(unit_vector_1, unit_vector_2)
  angle = np.arccos(dot_product)
  angle = np.rad2deg(angle)

  c = np.cross(vb, vv)
  if c>0:
      angle = -angle

  return angle

concat = io.StringIO('\\'.join([open(f).read() for f in glob(f"{dirname}/*")]))

names = ['ID', 'time', 'lng', 'lat','1','2','cap', 'speed', 'rank','dist_leader','3', 'dir_wind','4','5','start','leg','is_boat','vmc','vmg']

df = pd.read_csv(concat, sep='|', lineterminator='\\', header=0, names=names)
df = df.drop(columns=['1','2','3','4','5'])
real_start = df.start.value_counts().index[0]

df = df[df['start'] == real_start]
bouees = df[(df.is_boat == 0) & (df['rank'] == -1)]

df = df[df['rank'] > -1]
df['time'] = pd.to_datetime(df.time, unit='s')
df.cap = df.cap.apply(lambda x: x - 90)
df['angle_wind_boat'] = df.apply(lambda x: angle_from_vector_to_vector(x['cap'], x['dir_wind']), axis=1)
df['bord'] = df.apply(lambda x: 'b' if x['angle_wind_boat'] > 0 else 't', axis=1)


['A0090' 'A0254' 'A0377' 'A0405' 'A0408' 'A0475' 'A0758' 'A0763']


In [21]:

import plotly.express as px
import plotly.graph_objs as go
#Noel A0708
#Briez A0183
#Ganivez A0535
#Swinyard A0354
#vonck A0309
# babord positif, tribord négatif
id_coureur = 'A0354'
no_leg = 1
leg = df[df['leg'] == no_leg].reindex()

coureur = leg[(leg['ID'] == id_coureur)]
nb_vir = len(coureur['bord'].ne(coureur['bord'].shift()).cumsum().value_counts(sort=False).tolist()) - 1
fig = go.Figure([
    go.Scatter(x=coureur.time, y=coureur.angle_wind_boat, name="angle", mode="lines"),
    go.Scatter(x=coureur.time, y=coureur.vmg, yaxis='y2', name="vmg", mode="lines")
])

layout = go.Layout(
    yaxis = go.layout.YAxis(title = 'Angle vent/bateau', showline=False, ticklen=5, tickcolor='black'),
    yaxis2 = go.layout.YAxis(title = 'Vitesse', overlaying="y", side="right", showline=False, ticklen=5, tickcolor='black'),
    title=f"Leg {no_leg} :: Coureur {id_coureur}, {nb_vir} virements",
    #height=(len(selection) + 1) * (100 if len(selection) > 4 else 150),
    
    showlegend=True,
    legend=dict(
      yanchor="bottom",
      xanchor="left",
      y=0,
      x=0,
    ),
  )

fig.update_layout(layout)
fig.update_xaxes(tickformat='%M:%S')
fig

Empty DataFrame
Columns: [ID, time, lng, lat, cap, speed, rank, dist_leader, dir_wind, start, leg, is_boat, vmc, vmg, angle_wind_boat, bord]
Index: []
